In [1]:
import matplotlib.pyplot as plt
import numpy as np
from custom_env.actions import Acciones
from ipywidgets import interact

# Definir la función de recompensa


def compute_reward(actual: float, descartados: int, action: Acciones) -> float:

    reward = 0.0
    if descartados > 0:
        reward -= (descartados**2) * 1.0
    else:
        reward += 60.0

    reward += (1-actual)*200

    match action:
        case Acciones.PERMITIR:
            reward += 5.0
        case Acciones.DENEGAR:
            reward -= 2.0
        case _:
            reward += 0.0

    return reward

    reward_values: float = compute_reward(
        actual, descartados, Acciones.PERMITIR)


"""
def create_plot(axes, fila, valores,recompensas, accion:Acciones):
    col: int = Acciones.action_to_int(accion)
    axes[fila][col].plot(valores, recompensas,
                         label=f'Descartados')
    axes[fila][col].set_xlabel('Ocupación actual')
    axes[fila][col].set_ylabel('Recompensa (reward)')
    axes[fila][col].set_title(f'{accion.name} con descartados, ocupacion')
    axes[fila][col].legend()
"""


def create_plot(axes, fila, actual, valores, recompensas, accion: Acciones):
    col: int = Acciones.action_to_int(accion)
    axes[fila, col].plot(valores, recompensas, label='Reward')
    axes[fila, col].set_xlabel('Descartados')
    axes[fila, col].set_ylabel('Recompensa')
    axes[fila, col].set_title(f'{accion.name} - Reward - {actual:.2f}')
    axes[fila, col].legend()
    axes[fila, col].grid(True)


def plot_reward(descartados_range=8, num=3):
    actual_values = np.linspace(0, 1, num)
    descartados_values = np.arange(0, descartados_range + 1)
    # assert actual_values.shape == descartados_values.shape

    fig, axes = plt.subplots(num, 2, figsize=(10, num * 3))
    axes = np.atleast_2d(axes)  # Asegurarse de que sea una matriz 2D

    for i, actual in enumerate(actual_values):
        recompensas_permitir: list[float] = [compute_reward(
            actual, d, Acciones.PERMITIR) for d in descartados_values]
        recompensas_denegar: list[float] = [compute_reward(
            actual, d, Acciones.DENEGAR) for d in descartados_values]
        create_plot(axes, i, actual, descartados_values,
                    recompensas_permitir, Acciones.PERMITIR)
        create_plot(axes, i, actual, descartados_values,
                    recompensas_denegar, Acciones.DENEGAR)

    plt.tight_layout()
    plt.show()


# Crear la interactividad
interact(plot_reward, descartados_range=(1, 20, 1), num=(2, 10, 1))

Cargado a las 2025-03-02 16:33:50.546016


interactive(children=(IntSlider(value=8, description='descartados_range', max=20, min=1), IntSlider(value=3, d…

<function __main__.plot_reward(descartados_range=8, num=3)>

In [43]:
from matplotlib import axes
from matplotlib.lines import Line2D


def re_ocu(actual: float, coef=55) -> float:
    reward = 0.0
    reward += (1-actual)*coef
    return reward


def re_desca(descartados: float, coef=1.0) -> float:
    reward = 0.0
    if descartados > 0:
        reward -= (descartados**2) * coef
    else:
        reward += 60.0
    return reward


def re_accion(action: Acciones) -> float:
    reward = 0.0
    match action:
        case Acciones.PERMITIR:
            reward += 5.0
        case Acciones.DENEGAR:
            reward -= 2.0
        case _:
            reward += 0.0
    return reward


def plot_subplot(ax: axes._axes.Axes, datos_actual: list[float], recompensa_actual_in: list[float], datos_descartados: list[int], recompensa_descartados_in: list[float], accion: Acciones):

    recompensa_actual = list(map(lambda x: x + re_accion(accion), recompensa_actual_in))
    recompensa_descartados = list(map(lambda x: x + re_accion(accion), recompensa_descartados_in))


    line1: list[Line2D] = ax.plot(datos_actual, recompensa_actual,
                            label='Ocupación', color='blue')
    
    ax.set_xlabel('Ocupación')
    ax.set_ylabel('Recompensa')
    ax.spines['bottom'].set_color('blue')
    ax.xaxis.label.set_color('blue')
    ax.tick_params(axis='x', colors='blue')

    ax2: axes.Axes = ax.twiny()
    line2: Line2D = ax2.plot(
        datos_descartados, recompensa_descartados, label='Descartados', color='red')
    ax2.set_xlabel('Descartados')
    ax2.spines['top'].set_color('red')
    ax2.xaxis.label.set_color('red')
    ax2.tick_params(axis='x', colors='red')

    min_rew: float = min(recompensa_actual + recompensa_descartados)
    max_rew: float = max(recompensa_actual + recompensa_descartados)
    ax.set_ylim(min_rew, max_rew)
    ax2.set_ylim(min_rew, max_rew)

    lines: list[Line2D] = [*line1, *line2]
    labels: list[object] = [line.get_label() for line in lines]
    ax.legend(lines, labels)
    ax.set_title(accion.name)


def generate_subplots(axes, fila, datos_actual, datos_descartados, recompensa_actual, recompensa_descartados):
    plot_subplot(axes[fila, 0], datos_actual, recompensa_actual,
                 datos_descartados, recompensa_descartados, Acciones.PERMITIR)
    plot_subplot(axes[fila, 1], datos_actual, recompensa_actual,
                 datos_descartados, recompensa_descartados, Acciones.DENEGAR)


def generate_plot(coef_actual, coef_descartados, num_descartados=2):
    datos_actual = np.linspace(0, 1, num_descartados)
    datos_descartados = np.arange(0, num_descartados + 1)

    recompensa_actual = [re_ocu(a, coef=coef_actual) for a in datos_actual]
    recompensa_descartados = [re_desca(d, coef=coef_descartados) for d in datos_descartados]
    filas = 1
    fig, axes = plt.subplots(1, 2, figsize=(10, filas * 4))
    axes = np.atleast_2d(axes)

    generate_subplots(axes, 0, datos_actual, datos_descartados,
                      recompensa_actual, recompensa_descartados)

    plt.tight_layout()
    plt.show()


interact(generate_plot, coef_actual=(-100, 100, 5),
         coef_descartados=(1, 200, 5), num_descartados=(2, 10, 1))

interactive(children=(IntSlider(value=0, description='coef_actual', min=-100, step=5), IntSlider(value=96, des…

<function __main__.generate_plot(coef_actual, coef_descartados, num_descartados=2)>

In [119]:
from gymnasium.spaces import Box, Dict, Discrete

observation_space = Dict({
            "OcupacionCola": Box(low=0, high=1, dtype=np.float32),
            "Descartados": Box(low=0, high=np.inf, dtype=np.int64),  
        })
print(observation_space.values())
observation_space.sample()

ValuesView(Dict('Descartados': Box(0, 9223372036854775807, (1,), int64), 'OcupacionCola': Box(0.0, 1.0, (1,), float32)))


{'Descartados': array([0], dtype=int64),
 'OcupacionCola': array([0.37670204], dtype=float32)}